## Full Body Trajectory for Walking
**Implmentation of an A2C Agent for learning Biped Gait** 

Importing Required Libraries

In [1]:
import pybullet
import time
import pybullet_data
import numpy as np
import matplotlib.pyplot as plt

import gym
from gym import spaces

from qibullet import SimulationManager
from qibullet import NaoVirtual

GYM Based Environment

In [23]:
class NAO(gym.Env):
    #metadata = {'render.modes': ['human']}

    def __init__(self,imu_freq = 240,max_episode_length = 10000):
        super(NAO, self).__init__()
        # Define action and observation space
        self.action_space = spaces.Box(low=np.array([-1.53589,-1.14529,-0.379435,-0.0923279,-1.18944,-0.397761, 
                                                    -1.53589,-1.14529,-0.79046,-0.0923279,-1.1863,-0.768992]),
                                       high=np.array([0.48398,0.740718,0.79046,2.11255,0.922581,0.768992,
                                                     0.48398,0.740718,0.379435,2.11255,0.932006,0.397761]))
        self.observation_space = spaces.Box(low=np.array([-10,-10,-10,-10,-10,-10]), 
                                            high=np.array([10,10,10,10,10,10])) 
        """""""""""""""""""""
            Action Space: 12 component for lower limb joint positions
                          LHipPitch,LHipYawPitch,LHipRoll,LKneePitch,LAnklePitch,LAnkleRoll
                          RHipPitch,RHipYawPitch,RHipRoll,RKneePitch,RAnklePitch,RAnkleRoll
        """"""""""""""""""
            Observation Space: LPelvis,RPelvis Poition (z component), 
                               Torso Orientation (y component),
                               Toros's linear velocity (normal x & y component)
        """""""""""""""""""""

        #Instantiate Phisycs Client
        self.phisycsClient = pybullet.connect(pybullet.GUI)
        pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
        
        self.previous_config = np.array([0]*12)
        self.max_steps = max_episode_length
        self.steps = 0
        self.imu_freq = imu_freq
        
        self.velocity_x = 0
        self.simulation_manager = None
        self.imu = None
        self.nao = None
        self.reset()
        self.effort = 1.0
        
    def step(self,action):
        self.take_action(action)
        obs = self.observe()
        reward = self.calculate_reward(obs,action)
        done = self.is_terminated(obs)
        return obs, reward, done, {}
    
    def take_action(self,action):
        self.previous_config = self.nao.getAnglesPosition(["LHipPitch","LHipYawPitch","LHipRoll","LKneePitch","LAnklePitch","LAnkleRoll",
                                                           "RHipPitch","RHipYawPitch","RHipRoll","RKneePitch","RAnklePitch","RAnkleRoll"])

        self.nao.setAngles("LHipPitch",float(action[0]),self.effort)
        self.nao.setAngles("LHipYawPitch",float(action[1]),self.effort)
        self.nao.setAngles("LHipRoll",float(action[2]),self.effort)
        self.nao.setAngles("LKneePitch",float(action[3]),self.effort)
        self.nao.setAngles("LAnklePitch",float(action[4]),self.effort)
        self.nao.setAngles("LAnkleRoll",float(action[5]),self.effort)
        self.nao.setAngles("RHipPitch",float(action[6]),self.effort)
        self.nao.setAngles("RHipYawPitch",float(action[7]),self.effort)
        self.nao.setAngles("RHipRoll",float(action[8]),self.effort)
        self.nao.setAngles("RKneePitch",float(action[9]),self.effort)
        self.nao.setAngles("RAnklePitch",float(action[10]),self.effort)
        self.nao.setAngles("RAnkleRoll",float(action[11]),self.effort)
        pybullet.stepSimulation()
        pass
    
    def calculate_reward(self,obs,action):
        velx,yaw,pitch,roll,height,x = obs
        print (velx," ",yaw," ",pitch," ",roll," ",height)
        deltaTheta = action - self.previous_config
        reward = velx - yaw * 10 - pitch * 10 - roll * 10 - (height - 0.25)**2 + x - np.sum(deltaTheta) * 180 / np.pi
        return reward
    
    def observe(self):
        _, linear_acceleration = self.nao.getImuValues()
        a = linear_acceleration[0]
        self.velocity_x += (1/self.imu_freq) * a 
        
        torsoYaw = pybullet.getEulerFromQuaternion(self.nao.getLinkPosition("torso")[1])[2]
        torsoPitch = pybullet.getEulerFromQuaternion(self.nao.getLinkPosition("torso")[1])[1]
        torsoRoll = pybullet.getEulerFromQuaternion(self.nao.getLinkPosition("torso")[1])[0]
        
        Lpelvis,_ = self.nao.getLinkPosition("LPelvis")
        Rpelvis,_ = self.nao.getLinkPosition("RPelvis")
        pelvisHeight = 0.5*(Lpelvis[2] + Rpelvis[2])  
        
        x, y, theta = self.nao.getPosition()
        
        return np.array([self.velocity_x,torsoYaw , torsoPitch, torsoRoll, pelvisHeight,x])
    
    def is_terminated(self,obs):
        if self.steps == self.max_steps:
            self.steps = 0
            self.reset()
            return True
        elif abs(obs[1]) > 15 *(np.pi/180) or abs(obs[2]) > 15 *(np.pi/180) or obs[4] < 0.2 or obs[4] > 0.37:
            self.reset()
            return True
        else:
            return False
    
    def reset(self):
        # Close Previous Environment
        pybullet.resetSimulation()
        
        #Spawn Robot
        simulation_manager = SimulationManager()
        simulation_manager.setGravity(self.phisycsClient, [0.0, 0.0, -9.81])
        nao = simulation_manager.spawnNao(self.phisycsClient, spawn_ground_plane=True) 
        pybullet.setRealTimeSimulation(0)
        imu = nao.getImu()
        nao.subscribeImu(frequency=self.imu_freq)
        self.velocity_x = 0
        self.previous_config = nao.getAnglesPosition(["LHipPitch","LHipYawPitch","LHipRoll","LKneePitch","LAnklePitch","LAnkleRoll",
                                                      "RHipPitch","RHipYawPitch","RHipRoll","RKneePitch","RAnklePitch","RAnkleRoll"])  
        self.nao = nao
        self.imu = imu 
        return self.observe()
    
    def __close__(self):
        self.simulation_manager.stopSimulation(self.phisycsClient)


A2C Agent

In [24]:
import gym
import coax
import optax
import haiku as hk
import jax.numpy as jnp
import jax
from numpy import prod

# pick environment
name = 'A2C'
env = NAO()
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}") 

def func_v(S, is_training):
    # custom haiku function
    value = hk.Sequential([
                          hk.Linear(20),
                          hk.Linear(20),
                          hk.Linear(1,w_init=jnp.zeros),jnp.ravel])
    return value(S)  # output shape: (batch_size,)

def func_pi(S, is_training):
    shared = hk.Sequential((
        hk.Linear(20), jax.nn.relu,
        hk.Linear(20), jax.nn.relu,
    ))
    mu = hk.Sequential((
        shared,
        hk.Linear(15), jax.nn.relu,
        hk.Linear(12, w_init=jnp.zeros),
        hk.Reshape(env.action_space.shape),
    ))
    logvar = hk.Sequential((
        shared,
        hk.Linear(15), jax.nn.relu,
        hk.Linear(12, w_init=jnp.zeros),
        hk.Reshape(env.action_space.shape),
    ))
    return {'mu': mu(S), 'logvar': logvar(S)}


# function approximators
v = coax.V(func_v, env)
pi = coax.Policy(func_pi, env)


# specify how to update policy and value function
vanilla_pg = coax.policy_objectives.VanillaPG(pi, optimizer=optax.adam(0.001))
simple_td = coax.td_learning.SimpleTD(v, optimizer=optax.adam(0.002))


# specify how to trace the transitions
tracer = coax.reward_tracing.NStep(n=5, gamma=0.9)
buffer = coax.experience_replay.SimpleReplayBuffer(capacity=256)


for ep in range(10):
    s = env.reset()

    for t in range(env.max_steps):
        a, logp = pi(s, return_logp=True)
        s_next, r, done, info = env.step(a)

        # add transition to buffer
        # N.B. vanilla-pg doesn't use logp but we include it to make it easy to
        # swap in another policy updater that does require it, e.g. ppo-clip
        tracer.add(s, a, r, done, logp)
        while tracer:
            buffer.add(tracer.pop())

        # update
        if len(buffer) == buffer.capacity:
            for _ in range(4 * buffer.capacity // 32):  # ~4 passes
                transition_batch = buffer.sample(batch_size=32)
                metrics_v, td_error = simple_td.update(transition_batch, return_td_error=True)
                metrics_pi = vanilla_pg.update(transition_batch, td_error)
                env.record_metrics(metrics_v)
                env.record_metrics(metrics_pi)

            buffer.clear()

        if done:
            break

        s = s_next
del env

Exception in thread Thread-123:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: getLinkState failed.



0.0   0.0   -0.0   0.0   0.2750000059604645
0.0   0.0002178809283563007   0.0066029222950995175   -0.004013079878460402   0.27488061785697937
0.0   0.0005608137183312375   0.013312924809818117   -0.00416572231654007   0.2746238261461258
0.0   0.0012138548728479923   0.01903360915154402   -0.0014249296965494955   0.2742355763912201
0.0   0.001063977718955071   0.023579772066215393   0.003319487826961197   0.2737632244825363
0.0   0.0018070969855704827   0.02638522156406312   0.008243845421876188   0.27312998473644257
0.0   0.0017166713376508   0.029340987830814114   0.011927397148066492   0.27243709564208984
0.0   0.0013901543069140426   0.03160973018021902   0.015512464948920666   0.2715630531311035
0.0   0.0004557240051379665   0.03610337990674362   0.01526069355183469   0.2705262452363968
0.0   -0.0008284331628001622   0.038890629406344676   0.014545925312568384   0.2693999856710434
0.0   -0.0018026659440749091   0.03981591100423919   0.011937179295102287   0.26814089715480804
0.0   

0.0   -0.13173617430677426   0.05349100761393085   -0.0030390513875965365   0.2443002536892891
0.0   -0.138108871783531   0.060153493664195555   -0.0030001279335576466   0.2445581927895546
0.0   -0.14533339084551172   0.06541022108139331   -0.004123881051023547   0.24484732747077942
0.0   -0.15405909262598605   0.06740395479216198   -0.006151958041771503   0.24525747448205948
0.0   -0.15981945981477277   0.06626062699266964   -0.004920450283918945   0.24540212005376816
0.0   -0.16434525746460557   0.06637705228737828   -0.0010505645483947586   0.2451438084244728
0.0   -0.16817048040826668   0.06543963491988886   -0.0010858219214633308   0.24501968175172806
0.0   -0.17168701908028333   0.06529199374413665   -0.0011930271455141046   0.24456793069839478
0.0   -0.16993969817220714   0.06404779319991213   0.0014058442588202717   0.24390307813882828
0.0   -0.17037250315782235   0.06305327877872957   0.0024608567199933817   0.24292690306901932
0.0   -0.1689102153293375   0.0593027977334425   

Exception in thread Thread-135:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: getLinkState failed.

[A2C|MainThread|TrainMonitor|INFO] ep: 1,	T: 120,	G: -3.34e+03,	avg_G: -3.34e+03,	t: 119,	dt: 388.546ms
Exception in thread Thread-147:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: getLinkState failed.



0.0   0.0   -0.0   0.0   0.2750000059604645
0.0   -0.0006748888164924686   0.003304356105799685   0.0003765675359139262   0.2748878598213196
0.0   -0.0010826677467490231   0.005924531714517045   0.0004109012933528044   0.2746802717447281
0.0   -0.0012128631218005493   0.012279879137394973   0.00012017006173172348   0.27435894310474396
0.0   -0.002720353111951038   0.012780301300224462   6.97554218115539e-05   0.2739166021347046
0.0   -0.002204252498189619   0.010698666128989492   8.647822607984741e-05   0.2733011692762375
0.0   -4.6085572175921765e-05   0.007120732466410311   0.00015797498045963922   0.27263110876083374
0.0   0.0003123478001388283   0.004482996346201427   -6.717622463195213e-05   0.27184437215328217
0.0   0.0011953744297679948   0.0003895631770560981   -0.003823555597226891   0.2709333598613739
0.0   0.001196771281681606   -0.0012951637732362172   -0.008001345705758561   0.2698429524898529
0.0   0.000835824289493405   0.00023837177262108842   -0.012204598295145567   0.

...

Exception in thread Thread-304:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: getLinkState failed.

[A2C|MainThread|TrainMonitor|INFO] ep: 8,	T: 838,	G: 3.35e+03,	avg_G: -1.62e+03,	t: 29,	dt: 344.151ms
Exception in thread Thread-316:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: getLinkState failed.



0.0   0.0   -0.0   0.0   0.2750000059604645
0.0   0.0002872004078991365   0.007978915443479269   -0.0001654470107720617   0.2748880535364151
0.0   0.0007882834085784155   0.018376546944694233   0.003076096425137469   0.27460506558418274
0.0   0.0012482198077569235   0.03067015694572165   0.006349938592565135   0.274176225066185
0.0   0.0016779645448857078   0.04200211045656924   0.009955703852874833   0.27365823090076447
0.0   0.0020901500031877307   0.05290893649223469   0.013222784766267419   0.27296848595142365
0.0   0.002511363014120555   0.06373800726177095   0.01640524893271497   0.27215833961963654
0.0   0.003256757343544939   0.07405324304017441   0.019175326077661446   0.2711765617132187
0.0   0.004038374874378749   0.08409049857303023   0.021263922488027547   0.27006274461746216
0.0   0.004215028795441812   0.09195327672645756   0.02229295233682342   0.2688291519880295
0.0   0.0035194740819253514   0.10261939778317804   0.01914882122710903   0.26743635535240173
0.0   0.003281

Exception in thread Thread-328:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: getLinkState failed.

[A2C|MainThread|TrainMonitor|INFO] ep: 9,	T: 866,	G: 2.89e+03,	avg_G: -1.12e+03,	t: 27,	dt: 340.437ms
Exception in thread Thread-340:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: getLinkState failed.



0.0   0.0   -0.0   0.0   0.2750000059604645
0.0   0.00017434896204735934   0.007172054784706702   0.0005659739888679906   0.2748546898365021
0.0   0.0006975924230806842   0.01787138519871246   0.005434366550195351   0.2746167778968811
0.0   0.001649356089667203   0.02698001993239717   0.011662207327457506   0.2742452472448349
0.0   0.002716496951959067   0.03836363045518703   0.017157873153399388   0.2737031728029251
0.0   0.003184467645792471   0.047723798308000956   0.020809295949253116   0.27306702733039856
0.0   0.003658214631297076   0.058429635667534785   0.024495188866033236   0.27225786447525024
0.0   0.0037618232347620666   0.06528581822495517   0.02755750307119565   0.27131350338459015
0.0   0.0042827747047090815   0.07513217805067937   0.030600658031928185   0.27024632692337036
0.0   0.004265560858568781   0.08621608283228623   0.031276872116742085   0.26903416216373444
0.0   0.004428378393168232   0.0954420567823563   0.03191369627256354   0.2676998972892761
0.0   0.0041656

Exception in thread Thread-352:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: getLinkState failed.



In [22]:
pybullet.disconnect()

Exception in thread Thread-111:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kassra/.local/lib/python3.6/site-packages/qibullet/imu.py", line 110, in _imuScan
    physicsClientId=self.getPhysicsClientId())
pybullet.error: Not connected to physics server.

